In [2]:
# Test database connection
import os
from dotenv import load_dotenv
from supabase import create_client, Client

load_dotenv()

# You'll get these from Supabase dashboard
SUPABASE_URL = os.environ.get("SUPABASE_URL", "")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY", "")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Test connection
try:
    result = supabase.table('articles').select("*").limit(1).execute()
    print("✅ Database connection successful!")
    print(f"Current articles count: {len(result.data)}")
except Exception as e:
    print(f"❌ Connection failed: {e}")

✅ Database connection successful!
Current articles count: 0


In [3]:
import requests
import json
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import arxiv
from pprint import pprint

In [4]:
# Function to save arXiv papers to database
def save_papers_to_db(papers: list) -> None:
    """Save a list of papers to Supabase"""
    for paper in papers:
        # Convert arxiv paper object to database format
        paper_data = {
            'arxiv_id': paper.entry_id.split('/')[-1],
            'title': paper.title,
            'authors': [str(author) for author in paper.authors],
            'abstract': paper.summary,
            'published_date': paper.published.date().isoformat(),
            'categories': paper.categories,
            'pdf_url': paper.pdf_url,
            'processing_status': 'metadata_only'
        }
        
        try:
            result = supabase.table('articles').insert(paper_data).execute()
            print(f"✅ Saved: {paper.title[:50]}...")
        except Exception as e:
            print(f"❌ Failed to save {paper.title[:30]}...: {e}")

# Test with some papers
search = arxiv.Search(query="cat:cs.AI AND all:fMRI", max_results=10)
papers = list(search.results())

print(f"Saving {len(papers)} papers to database...")
save_papers_to_db(papers)

# Verify they were saved
result = supabase.table('articles').select("*").execute()
print(f"\nDatabase now contains {len(result.data)} articles")

C:\Users\murph\AppData\Local\Temp\ipykernel_23968\746264699.py:25: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  papers = list(search.results())


Saving 10 papers to database...
✅ Saved: BrainMAP: Learning Multiple Activation Pathways in...
✅ Saved: Lite-Mind: Towards Efficient and Robust Brain Repr...
✅ Saved: Animate Your Thoughts: Decoupled Reconstruction of...
✅ Saved: Conscious Intelligent Systems - Part 1 : I X I...
✅ Saved: Uncovering communities of pipelines in the task-fM...
✅ Saved: GDAIP: A Graph-Based Domain Adaptive Framework for...
✅ Saved: MindFormer: Semantic Alignment of Multi-Subject fM...
✅ Saved: Source-Free Collaborative Domain Adaptation via Mu...
✅ Saved: Artificial Intelligence in Fetal Resting-State Fun...
✅ Saved: BrainCLIP: Bridging Brain and Visual-Linguistic Re...

Database now contains 10 articles
